In [1]:
!git clone https://github.com/nkt780426/Bachelor-s-Project.git

Cloning into 'Bachelor-s-Project'...
remote: Enumerating objects: 541, done.
remote: Counting objects: 100% (133/133), done.
remote: Compressing objects: 100% (96/96), done.
remote: Total 541 (delta 33), reused 101 (delta 28), pack-reused 408 (from 1)
Receiving objects: 100% (541/541), 77.04 MiB | 17.24 MiB/s, done.
Resolving deltas: 100% (34/34), done.


In [2]:
!mv Bachelor-s-Project/going_modular .

In [3]:
!ls -la

total 48
drwxr-xr-x 4 root root  4096 Dec 20 06:22 .
drwxr-xr-x 6 root root  4096 Dec 20 06:22 ..
drwxr-xr-x 5 root root  4096 Dec 20 06:22 Bachelor-s-Project
drwxr-xr-x 8 root root  4096 Dec 20 06:22 going_modular
---------- 1 root root 31491 Dec 20 06:22 __notebook__.ipynb


In [4]:
import warnings
warnings.filterwarnings("ignore")
import os

import torch
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
import albumentations as A

from going_modular.dataloader.magface import create_magface_dataloader
from going_modular.model.MagFaceRecognition import MagFaceRecognition
from going_modular.train_eval.magface.train import fit
from going_modular.loss.MagLoss import MagLoss
from going_modular.utils.MultiMetricEarlyStopping import MultiMetricEarlyStopping
from going_modular.utils.ModelCheckPoint import ModelCheckpoint
from going_modular.utils.transforms import RandomResizedCropRect, GaussianNoise

device = "cuda" if torch.cuda.is_available() else "cpu"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Đặt seed toàn cục
seed = 42
torch.manual_seed(seed)

CONFIGURATION = {
    # Thư mục
    'type': 'albedo',
    'train_dir': '/kaggle/input/hoangvn-3dfacerecognition/Albedo/train',
    'test_dir': '/kaggle/input/hoangvn-3dfacerecognition/Albedo/test',
    
    # Cấu hình train
    'backbone': 'inception-resnet-v1',
    'epochs': 500,
    'num_workers': 4,
    'batch_size': 16,
    'image_size': 224,
    'num_class': len(os.listdir('/kaggle/input/hoangvn-3dfacerecognition/Albedo/train')),
    'embedding_size': 512,
    
    'learning_rate': 0.2,
    'weight_decay': 5e-4,
    'momentum': 0.9,
    'alpha': 0.9,
    
    # Hàm m(ai) giúp thay đổi ai từ 0.25 đến 1.6
    'scale': 64,
    'lambda_g': 20,
    'l_margin': 0.45, 
    'u_margin': 0.8,
    'l_a': 10, 
    'u_a': 110,
}

train_transform = A.Compose([
    RandomResizedCropRect(256),
    GaussianNoise(),
])

test_transform = A.Compose([
    A.Resize(height=CONFIGURATION['image_size'], width=CONFIGURATION['image_size'])
])

train_dataloader, test_dataloader = create_magface_dataloader(CONFIGURATION, train_transform, test_transform)

model = MagFaceRecognition(CONFIGURATION).to(device)
criterion = MagLoss(conf = CONFIGURATION)

optimizer = torch.optim.Adam(model.parameters(), lr=CONFIGURATION['learning_rate'])
scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=40, T_mult=1, eta_min=1e-6)
checkpoint_path = os.path.abspath('checkpoint/magface/' + CONFIGURATION['type'] + '/models/checkpoint.pth')
modle_checkpoint = ModelCheckpoint(filepath=checkpoint_path, verbose=1)
earlystop_dir = os.path.abspath('checkpoint/magface/' + CONFIGURATION['type'] + '/models')
early_stopping = MultiMetricEarlyStopping(
    monitor_keys=['test_cosine_auc', 'test_cosine_accuracy', 'test_euclidean_auc', 'test_euclidean_accuracy'],
    patience=50,
    mode='max',
    verbose=1,
    save_dir=earlystop_dir,
    start_from_epoch=50
)            

In [5]:
fit(CONFIGURATION, 0, model, device, train_dataloader, test_dataloader, criterion, optimizer, scheduler, early_stopping, modle_checkpoint)

Epoch 1:
	train: loss 2.855 | loss id   2.83 | top_1_acc 0.0045 | top_5_acc 0.0236 | cos_auc: 0.5934 | cos_acc: 0.5967 | eu_auc: 0.5950 | eu_acc: 0.6161
	test: cos_auc: 0.6362 | cos_acc: 0.6010 | eu_auc: 0.6259 | eu_acc: 0.4910
	Saving model and optimizer state to /kaggle/working/checkpoint/magface/albedo/models/checkpoint.pth
Epoch 2:
	train: loss 0.631 | loss id   0.61 | top_1_acc 0.0056 | top_5_acc 0.0202 | cos_auc: 0.5559 | cos_acc: 0.4172 | eu_auc: 0.5587 | eu_acc: 0.3887
	test: cos_auc: 0.5985 | cos_acc: 0.5190 | eu_auc: 0.5718 | eu_acc: 0.4509
	Saving model and optimizer state to /kaggle/working/checkpoint/magface/albedo/models/checkpoint.pth
Epoch 3:
	train: loss 0.500 | loss id   0.48 | top_1_acc 0.0034 | top_5_acc 0.0236 | cos_auc: 0.5723 | cos_acc: 0.4019 | eu_auc: 0.5661 | eu_acc: 0.3759
	test: cos_auc: 0.6224 | cos_acc: 0.5405 | eu_auc: 0.6140 | eu_acc: 0.6771
	Saving model and optimizer state to /kaggle/working/checkpoint/magface/albedo/models/checkpoint.pth
Epoch 4:
	tra